In [15]:
# 📌 Sistema de Recomendação de Games (Steam Dataset)

# =============================
# 1. Importação das bibliotecas
# =============================
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# =============================
# 2. Carregamento do Dataset
# =============================
# ⚠️ O dataset é grande (~370MB).
# Sugiro baixar antes do HuggingFace e colocar no mesmo diretório.
# URL: https://huggingface.co/datasets/FronkonGames/steam-games-dataset

from google.colab import drive
drive.mount('/content/drive')
arquivo = "games.csv"   # renomeie conforme o nome baixado
df = pd.read_csv(arquivo, encoding="ISO-8859-1", engine="python", on_bad_lines="skip")
df.columns = df.columns.str.lower()
print(df.shape)
print(df.columns.tolist())


# =============================
# 3. Pré-processamento
# =============================

# Mantemos apenas colunas relevantes
df = df[['name', 'genres', 'positive', 'negative', 'average playtime forever', 'price', 'required age', 'score rank', 'notes']]

# Remove duplicados e valores ausentes
df.dropna(subset=['name', 'genres'], inplace=True)
df.drop_duplicates(subset=['name'], inplace=True)

# Cria um score de avaliação
df['score'] = df['positive'] / (df['positive'] + df['negative'] + 1)

# Reseta índice
df = df.reset_index(drop=True)

print("Registros após limpeza:", len(df))
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(40241, 39)
['appid', 'name', 'release date', 'estimated owners', 'peak ccu', 'required age', 'price', 'discountdlc count', 'about the game', 'supported languages', 'full audio languages', 'reviews', 'header image', 'website', 'support url', 'support email', 'windows', 'mac', 'linux', 'metacritic score', 'metacritic url', 'user score', 'positive', 'negative', 'score rank', 'achievements', 'recommendations', 'notes', 'average playtime forever', 'average playtime two weeks', 'median playtime forever', 'median playtime two weeks', 'developers', 'publishers', 'categories', 'genres', 'tags', 'screenshots', 'movies']
Registros após limpeza: 3629


,name,genres,positive,negative,average playtime forever,price,required age,score rank,notes,score
0,"Oct 21, 2008","Casual,Indie,Sports",6.0,11.0,0.0,0,19.99,NaN,NaN,0.333333
1,"Oct 12, 2017","Action,Indie",53.0,5.0,0.0,0,0.99,NaN,NaN,0.898305
2,"Nov 17, 2021","Action,Adventure,Indie,Strategy",0.0,0.0,0.0,0,4.99,NaN,NaN,0.000000
3,"Jul 23, 2020","Adventure,Casual,Indie",3.0,0.0,0.0,0,5.99,NaN,NaN,0.750000
4,"Feb 3, 2020","Adventure,Indie",50.0,8.0,0.0,0,0.00,NaN,This Game may contain content not appropriate ...,0.847458


In [17]:
# =============================
# 4. Matriz de Similaridade (baseada nos gêneros)
# =============================

# Vetorização dos gêneros
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df['genres'])

# Similaridade do cosseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("Matriz de similaridade pronta:", cosine_sim.shape)

Matriz de similaridade pronta: (3629, 3629)


In [18]:
# =============================
# 5. Função de Recomendação
# =============================

# Mapeia nome -> índice
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

def recomendar_jogos(nome_jogo, n=5):
    if nome_jogo not in indices:
        return [f"❌ Jogo '{nome_jogo}' não encontrado na base."]

    idx = indices[nome_jogo]

    # Pega as similaridades do jogo escolhido
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordena pelo score de similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Ignora o primeiro (ele mesmo) e pega top N
    sim_scores = sim_scores[1:n+1]

    # Pega os índices dos jogos recomendados
    game_indices = [i[0] for i in sim_scores]

    return df[['name', 'genres', 'score', 'price']].iloc[game_indices]


In [22]:
# =============================
# 6. Testes do Sistema
# =============================

print("🎮 Recomendações para 'Counter-Strike':")
print(recomendar_jogos("Oct 21, 2008"))

print("\n🎮 Recomendações para 'The Elder Scrolls V: Skyrim':")
print(recomendar_jogos("Nov 17, 2021"))

print("\n🎮 Recomendações para 'Dota 2':")
print(recomendar_jogos("Feb 3, 2020"))


🎮 Recomendações para 'Counter-Strike':
              name               genres     score  price
285   Jun 28, 2017  Casual,Indie,Sports  0.666667      0
855   Nov 14, 2019  Casual,Indie,Sports  0.000000      0
1036  Apr 10, 2020  Casual,Indie,Sports  0.600000      0
1309  Oct 16, 2021  Casual,Indie,Sports  0.000000      0
1749  May 29, 2022  Casual,Indie,Sports  0.000000      0

🎮 Recomendações para 'The Elder Scrolls V: Skyrim':
              name                           genres     score  price
206   Sep 29, 2016  Action,Adventure,Indie,Strategy  0.761905      0
265    Jan 6, 2020  Action,Adventure,Indie,Strategy  0.564103      0
291   May 31, 2020  Action,Adventure,Indie,Strategy  0.500000      0
695   Apr 16, 2021  Action,Adventure,Indie,Strategy  0.333333      0
1091  Nov 21, 2019  Action,Adventure,Indie,Strategy  0.739130      0

🎮 Recomendações para 'Dota 2':
            name           genres     score  price
32  Mar 22, 2019  Adventure,Indie  0.615385      0
51  Oct 28, 2020  